# [Ateliers: Technologies de l'intelligence Artificielle](https://github.com/wikistat/AI-Frameworks)

<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 
<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" width=400, style="max-width: 150px; display: inline"  alt="Wikistat"/></a>
<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" width=400,  style="float:right;  display: inline" alt="IMT"/> </a>
    
</center>

# Data : Cdiscount's product description.

This dataset has been released from Cdiscount for a data competition (type kaggle) on the french website [datascience.net](https://www.datascience.net/fr/challenge). <br>
The test dataset of this competition has not been released, so we used a subset of 1M producted of the original train dataset(+15M rows) all along the **Natural Language Processing** lab.<br>
The objective of this competition was to classify the text description of various product into various categories that compose the navigation tree of Cdiscount website. It is composed of 4,733 categories organized within 44 meta categories. <br>

The objective of this lab is not win the competition so we will only used the meta-categories.

# Part 2 : Words embedding. Application to text classification and semi-supervised learning.

In this first notebook we study three words embedding methods:

* [Word2Vec](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)
* [FastText](https://arxiv.org/pdf/1607.04606.pdf)
* [Glove](https://nlp.stanford.edu/pubs/glove.pdf)

We for each of these three method we will:

* Study their characteristics
* Explore the embedding they produces
* Check how they perform on classification problems
* Check how they can overcome problem with few labeled data.

## Téléchargement des librairies

In [9]:
#Importation des librairies utilisées
import time
import pandas as pd
import numpy as np
import collections
import itertools
import os
import warnings
warnings.filterwarnings('ignore')

import sklearn.model_selection as sms
from solution.clean import CleanText

## Load Data

We download the train and test data and generate the same cleaned columns and the same train/validation split than part 1

In [10]:
ct = CleanText()
data = pd.read_csv("data/cdiscount_train.csv.zip",sep=",", nrows=1000)
ct.clean_df_column(data, "Description", "Description_cleaned")
print("The train dataset is composed of %d lines" %data.shape[0])
data.head(5)

 17%|█▋        | 173/1000 [00:00<00:00, 1728.48it/s]

Start Clean 1000 lines


100%|██████████| 1000/1000 [00:00<00:00, 2055.64it/s]

The train dataset is composed of 1000 lines


,Categorie1,Categorie2,Categorie3,Description,Libelle,Marque,Description_cleaned
0,INFORMATIQUE,CONNECTIQUE - ALIMENTATION,BATTERIE,Batterie Acer Aspire One 751H-52Yr - Li-Ion 11...,Batterie Acer Aspire One 751H-52Yr,AUCUNE,batter acer aspir one h yr li ion v mah wh noi...
1,TELEPHONIE - GPS,ACCESSOIRE TELEPHONE,COQUE - BUMPER - FACADE TELEPHONE,Coque rigide Bleu lagon pour ALCATEL OT / 6033...,Coque rigide Bleu lagon pour ALCATEL OT / 6033 …,MUZZANO,coqu rigid bleu lagon alcatel ot motif drapeau...
2,TELEPHONIE - GPS,ACCESSOIRE TELEPHONE,COQUE - BUMPER - FACADE TELEPHONE,Facades et coques CELLULAR LINE SHCKGALS 3 MIN...,Facades et coques CELLULAR LINE SHCKGALS 3 MINIP,CELLULAR LINE,facad coqu cellular lin shckgal minip marqu ag...
3,TELEPHONIE - GPS,ACCESSOIRE TELEPHONE,COQUE - BUMPER - FACADE TELEPHONE,Coque meteore TPU LG Nexus 4 / E960,Coque meteore TPU LG Nexus 4 / E960,AUCUNE,coqu meteor tpu lg nexus e
4,TELEPHONIE - GPS,ACCESSOIRE TELEPHONE,COQUE - BUMPER - FACADE TELEPHONE,Coque souple Transparente pour LG G FLEX D959 ...,Coque souple Transparente pour LG G FLEX D959 m…,MUZZANO,coqu soupl transparent lg g flex motif keep ca...


In [11]:
data_train, data_valid = sms.train_test_split(data, test_size=0.1, random_state=42)

In [12]:
data_test = pd.read_csv("data/cdiscount_test.csv.zip",sep=",")
ct.clean_df_column(data_test, "Description", "Description_cleaned")
print("The train dataset is composed of %d lines" %data_test.shape[0])
data_test.head(5)

  0%|          | 177/50000 [00:00<00:28, 1760.98it/s]

Start Clean 50000 lines


100%|██████████| 50000/50000 [00:22<00:00, 2202.96it/s]


The train dataset is composed of 50000 lines


,Categorie1,Categorie2,Categorie3,Description,Libelle,Marque,Description_cleaned
0,BRICOLAGE - OUTILLAGE - QUINCAILLERIE,ELECTRICITE  DOMOTIQUE,MULTIPRISE - RALLONGE - ENROULEUR,Rallonge CEE avec inverseur de phase 25 m 16 A...,Rallonge CEE avec inverseur de phase 25 m 16 A,AUCUNE,rallong ce inverseur phas rallong ce haut qual...
1,DECO - LINGE - LUMINAIRE,OBJET DE DECORATION - BIBELOT,BUSTE - MANNEQUIN,Sun d’koh - Buste de Bouddha sur socle - cimen...,Sun d’koh - Buste de Bouddha sur socle - ciment…,SUN D’KOH,sun dkoh bust bouddh socl ciment cm tet bouddh...
2,HYGIENE - BEAUTE - PARFUM,NAIL ART,STICKERS - AUTOCOLLANT - STRASS - PAILLETTES -...,La planche contient 24 motifs Support : ongle ...,STICKERS COLLIER STRASS FLEUR ONGLE ADHESIF,AUCUNE,planch contient motif support ongle naturel ca...
3,LIBRAIRIE,AUTRES LIVRES,AUTRES LIVRES,De Blodwenn Mauffret aux éditions IBIS ROUGE,LE CARNAVAL DE CAYENNE,AUCUNE,blodwen mauffret edit ibis roug
4,VETEMENTS - LINGERIE,ACCESSOIRE MODE,CHAPEAU - BOB,Chapeau - Casquette béret Modèle féminin et t...,Chapeau ... TU,AUCUNE,chapeau casquet beret model feminin tendanc mo...


## Word2Vec

In this part, we will generate`Word2Vec` model thanks to the [**gensim**](https://radimrehurek.com/gensim/index.html) python library.

In [13]:
import gensim

### Build Word2Vec model

La fonction `gensim.models.Word2Vec`qui permet de construire des modèle Word2Vec prend en entrée une liste de tokens. On tranformer donc nos données dans un premier temps

In [ ]:
train_array_token = [line.split(" ") for line in train_array]
valid_array_token = [line.split(" ") for line in valid_array]
train_array_token[0]

Cette fonction contient un grand nombre d' [arguments](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec). Le but de ce TP n'est pas d'optimiser les paramètres de ce modèle mais de les comprendre. Nous allons donc fixer quelques arguments par défault : 

* Features_dimension = 300 : Dimension de l'espace des features (d'*embedding*) qui sera crée.
* hs = 0
* negative = 10

**Q** A quoi servent les arguments *hs* et *negative*? Quels influences ces arguments ont sur le modèle avec les valeurs définies ici?

In [ ]:
Features_dimension = 300
hs = 0
negative = 10

Nous allons créer deux modèles :

* Un modèle **skip-sgram**, sg = 1
* Un modèle **CBOW**, sg = 0

In [ ]:
sg = 1
print("Start learning skip-gram Word2Vec")
ts = time.time()
model_sg = gensim.models.Word2Vec(train_array_token, sg=sg, hs=hs, negative=negative, min_count=1, size=Features_dimension)
te = time.time()
t_learning = te-ts
print("Learning time : %.2f seconds Word2Vec" %t_learning)


sg = 0
print("Start learning CBOW Word2Vec")
ts = time.time()
model_cbow = gensim.models.Word2Vec(train_array_token, sg=sg, hs=hs, negative=negative, min_count=1, size=Features_dimension)
te = time.time()
t_learning = te-ts
print("Learning time : %.2f seconds Word2Vec" %t_learning)




**Q** Que dire du temps d'apprentissage de ces deux modèles? D'ou vient cette différence?

### Pre-Trained Model

Comme pour les réseaux de convolution, des modèles pré-entrainés de Word2Vec éxistent également. 
Le plus célèbre et le plus utilisé étant [`GoogleNewsVectors`](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit) appris sur plus de 100 milliard de mots à partir des articles de GoogleNews. Cependant ce modèle est en anglais, et n'est donc )as utile ici.


On utilisera des modèle appris dans le projet suivant [https://github.com/Kyubyong/wordvectors](https://github.com/Kyubyong/wordvectors) appris sur 1Giga d'articles de wikipedia en mode **Skip-Gram*

Vous pouvez télécharger ce modèle en suivant ce [lien](https://drive.google.com/file/d/0B0ZXk88koS2KM0pVTktxdG15TkE/view). Dezipez-le puis téléchargez le modèle en indiquant la direction du fichier "fr/bin"

In [ ]:
model_online_dir = "data/fr/fr.bin"
#model_online_dir = "ACOMPLETER/fr.bin"
model_online = gensim.models.Word2Vec.load(model_online_dir)

### Propriété du modèle

Nous allons maintenant comparer quelques propriétés de chacun des trois modèles à notre disposition (*CBOW*, *Skip-Gram* et le modèle *online*)

Les modèles que nous avons appris l'ont été sur les mots *racinisé*. Ainsi, nous allons avoir besoin de la racine des mots pour tester les différentes propriétés du modèle.

In [ ]:
import nltk 
stemmer=nltk.stem.SnowballStemmer('french')

#### Most similar world

La fonction `most_similar` de **gensim** permet de retrouver les mots les plus proches à un ou une combinaison de mots données en argument.

**Q** A l'aide de la [documentation](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.WordEmbeddingsKeyedVectors.most_similar) répondez aux questions suivantes. Quelle est la mesure de similarité utilisée? Dans quel espace est-elle utilisé? Comment fonctionne la fonction lorsque plusieurs mots sont passés en paramètres? 

##### 1 mot


**Exercice** Pour chacun de ces trois modèles, affichez les sorties de la fonction 'most_similar' pour le mot *homme*. 

In [ ]:
# %load solution/2_3.py

**Q** Comparez la qualité de prévision des modèles que nous avons entrainés sur le jeu de données 'Cdiscount' avec celui appris online. Que pouvez-vous en dire? 

**Q** Comparez les prévisions des deux modèles que nous avons entrainés. Que pouvez-vous en dire?

**Exercice** Affichez maintenant les sorties de la fonction 'most_similar' pour le mot *femme*. 

**Exercice** Affichez les sorties de la fonction 'most_similar' pour des mots propre au jeu de données (ex. *xbox*, *pantalon*,..)

##### Combinaison de mots

**Exercice** Pour chacun de ces trois modèles, affichez les sorties de la fonction 'most_similar' pour l'opération suivante : *roi* + *femme* - *homme* à l'aide des arguments *positive* et *negative* de la fonction. Commentez encore une fois la qualité de sortie des différents modèle

In [ ]:
# %load solution/2_4.py

**Exercice** Testez d'autres combinaisons si vous le souhaitez.

### Predict output word

La fonction `predict_output_word` de **gensim** permet de retrouver les mots prédit par le modèle à partir d'un mot ou d'une combinaison de mots données en argument.

**Exercice**  Affichez la prédiction des trois modèles pour des mots/combinaisons de mots communs (*homme*, *femme*) ou plus propre au jeu de données étudiés. (*coque*-*de*-*téléphone*)...

In [ ]:
# %load solution/2_5.py

### Build Features

Nous allons maintenant créer des matrices de features à partir des modèles de **Word2Vec** générés précédemment dans un but de prédiction. 

La modèles créés permettent d'obtenir pour chaque mot `x`, sa représentation dans l'espace d'embeddings de la manière suivante : 

*x_feature = model_word2vec[x]* 

Dans notre problématique, les individus que nous cherchons à classer sont des descriptions représentées par une liste de token à l'issue du nettoyage du calepin précédent.  Il exsite donc plusieurs façon de représenter ces individus à l'aide de modèle **Word2Vec**. 

1. Moyenne des vecteurs dans l'espace des features des différents mots/token de la description.
2. Moyenne pondérées des vecteurs dans l'espace des features des différents mots/token de la description en fonction de l'occurence de chacun de ces mots/tokens dans la description.
3. Moyenne pondérées par des poids calculés à l'aide du TF-IDF.
4. etc...

C'est la seconde solution qui sera utilisée ici. 

Les fonctions détaillées ci-dessous permettent de :
    
* `get_features_mean` : retourne le vecteur moyen dans l'espace d'embedding, des projections des mots/tokens composant *lines*
* `get_matrix_features_means` : applique la fonction `get_features_mean` sur tous les éléments de la matrice *X*.

In [ ]:
def get_features_mean(lines, model, f_size):
    features = [model[x] for x  in lines if x in model]
    if features == []:   
        fm =np.ones(f_size)
    else :
        fm = np.mean(features,axis=0)
    return fm

def get_matrix_features_means(X, model, f_size):
    X_embedded_ = list(map(lambda x : get_features_mean(x, model, f_size), X))
    X_embedded = np.vstack(X_embedded_)
    return X_embedded

#### Cbow

In [ ]:
ts = time.time()
X_embedded_train_cbow = get_matrix_features_means(train_array_token, model_cbow, Features_dimension)
te = time.time()
t_build = te-ts
#np.save(embedded_train_dir, X_embedded_train)
print("Time conversion : %d seconds"%t_build)
print("Shape Matrix : (%d,%d)"%X_embedded_train_cbow.shape)
np.save(DATA_OUTPUT_DIR +"/embedded_train_cbow", X_embedded_train_cbow)

ts = time.time()
X_embedded_valid_cbow = get_matrix_features_means(valid_array_token, model_cbow, Features_dimension)
te = time.time()
t_build = te-ts
#np.save(embedded_train_dir, X_embedded_train)
print("Time conversion : %d seconds"%t_build)
print("Shape Matrix : (%d,%d)"%X_embedded_valid_cbow.shape)
np.save(DATA_OUTPUT_DIR +"/embedded_valid_cbow", X_embedded_valid_cbow)

#### Skip-Gram

In [ ]:
ts = time.time()
X_embedded_train_sg = get_matrix_features_means(train_array_token, model_sg, Features_dimension)
te = time.time()
t_build = te-ts
#np.save(embedded_train_dir, X_embedded_train)
print("Time conversion : %d seconds"%t_build)
print("Shape Matrix : (%d,%d)"%X_embedded_train_sg.shape)
np.save(DATA_OUTPUT_DIR +"/embedded_train_sg", X_embedded_train_sg)

ts = time.time()
X_embedded_valid_sg = get_matrix_features_means(valid_array_token, model_sg, Features_dimension)
te = time.time()
t_build = te-ts
#np.save(embedded_train_dir, X_embedded_train)
print("Time conversion : %d seconds"%t_build)
print("Shape Matrix : (%d,%d)"%X_embedded_valid_sg.shape)
np.save(DATA_OUTPUT_DIR +"/embedded_valid_sg", X_embedded_valid_sg)

#### Online model

In [ ]:
data_valid_clean = pd.read_csv("data/cdiscount_valid_clean.csv").fillna("")
data_train_clean = pd.read_csv("data/cdiscount_train_clean.csv").fillna("")

train_array_token_nostem = [line.split(" ") for line in data_train_clean["Description"].values]
valid_array_token_nostem = [line.split(" ") for line in data_valid_clean["Description"].values]

In [ ]:
ts = time.time()
X_embedded_train_online = get_matrix_features_means(train_array_token_nostem, model_online, Features_dimension)
te = time.time()
t_build = te-ts
#np.save(embedded_train_dir, X_embedded_train)
print("Time conversion : %d seconds"%t_build)
print("Shape Matrix : (%d,%d)"%X_embedded_train_online.shape)
np.save(DATA_OUTPUT_DIR +"/embedded_train_online", X_embedded_train_online)

ts = time.time()
X_embedded_valid_online = get_matrix_features_means(valid_array_token_nostem, model_online, Features_dimension)
te = time.time()
t_build = te-ts
#np.save(embedded_train_dir, X_embedded_train)
print("Time conversion : %d seconds"%t_build)
print("Shape Matrix : (%d,%d)"%X_embedded_valid_online.shape)
np.save(DATA_OUTPUT_DIR +"/embedded_valid_online", X_embedded_valid_online)